## 1. Library Import

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import pickle
import random

from tqdm.notebook import tqdm
tqdm.pandas()

import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling

from sklearn.model_selection import train_test_split

In [2]:
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']= '0'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('Device:', device)  # 출력결과: cuda 
print('Count of using GPUs:', torch.cuda.device_count()) 
print('Current cuda device:', torch.cuda.current_device()) 

Device: cuda
Count of using GPUs: 1
Current cuda device: 0


In [3]:
seed = 42

def set_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False # for faster training, but not deterministic

set_seeds(seed)

## 2. Hyper-Parameter

In [4]:
weight_path = 'beomi/kcbert-base'
max_len = 300
epochs = 20
batch_size = 32

## 3. Data Load

In [5]:
law_data = pd.read_csv('./data/law_fp.csv')

del law_data['Unnamed: 0']

In [6]:
law_data

,subdomain,pre_content,content,label
0,민법,그 당시 알 수 있었거나 또는 알고서 이를 주장하지 않았던 사항에 한하여서만 기판력...,그 당시 알 수 있었거나 또는 알고서 이를 주장하지 않았던 사항에 한하여서만 기판력...,0
1,민법,이러한 흐름은 표현의 자유에 대하여 강한 보호를 부여해 온 미국의 전통과 관련이 깊...,이러한 흐름은 표현의 자유에 대하여 강한 보호를 부여해 온 미국의 전통과 관련이 깊...,0
2,민법,양도 제한 조건부 주식은 임직원에게 일정 기간 동안 처분이 금지되는 주식을 지급하는...,양도 제한 조건부 주식은 임직원에게 일정 기간 동안 처분이 금지되는 주식을 지급하는...,0
3,민법,대법원도 판례 과 판례 에서 이와 동일하게 판단하였다,"대법원도 판례 1과 판례 3에서, 이와 동일하게 판단하였다.",0
4,민법,제 조 제 조에서의 해제를 하면 증여 계약은 처음부터 절대적으로 무효가 되고 이러한...,"제555조∼제558조에서의 해제를 하면, 증여 계약은 처음부터 절대적으로 무효가 되...",0
...,...,...,...,...
99995,형법,진입하기 전 자동조타에서 수동 조타로 바꾸었다는 것으로도 짐작 가능하다,진입하기 전 자동조타에서 수동 조타로 바꾸었다는 것으로도 짐작 가능하다.,4
99996,형법,자수범과 의무범의 경우에는 적용될 수 없음을 논의의 전제로서 못박고 있다,자수범과 의무범의 경우에는 적용될 수 없음을 논의의 전제로서 못박고 있다.,4
99997,형법,이를 킨트호이저의 말을 빌어 표현하면 정범은 그 사건을 형법적 의미에서 그 사람의 ...,"이를 킨트호이저의 말을 빌어 표현하면, 정범은 그 사건을 형법적 의미에서 그 사람의...",4
99998,형법,한편 낙태행위의 시점까지 태아가 생존하고 있지 않으면 안되고 이미 사망한 태아는 낙...,"한편 낙태행위의 시점까지 태아가 생존하고 있지 않으면 안되고, 이미 사망한 태아는 ...",4


## 4. Data Split

In [7]:
# train_pretrain / fine_tuning split
train_data, val_data, train_label, val_label = train_test_split(law_data[law_data.columns[:-1]],
                                                                law_data['label'],
                                                                test_size = 0.2,
                                                                random_state = 42)

In [8]:
train_label.value_counts()

4    16022
3    16013
2    15998
1    15985
0    15982
Name: label, dtype: int64

In [9]:
val_label.value_counts()

0    4018
1    4015
2    4002
3    3987
4    3978
Name: label, dtype: int64

In [10]:
law_train = train_data.copy()
law_train['label'] = train_label

law_val = val_data.copy()
law_val['label'] = val_label

In [11]:
print(len(law_train), len(law_val))

80000 20000


In [12]:
law_train.to_csv('./data/law_fp_train.csv')
law_val.to_csv('./data/law_fp_val.csv')

## 6. MLM(Further Pre-training)

In [13]:
mlm_model = AutoModelForMaskedLM.from_pretrained(weight_path)
tokenizer = AutoTokenizer.from_pretrained(weight_path)

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### (1) MLM 형태로 변환

In [14]:
def tokenize_function(sentence):
    text = tokenizer(sentence, add_special_tokens=True, max_length=max_len, padding='max_length', truncation=True)
    text['labels'] = text['input_ids'].copy()
    return text

In [15]:
law_train['token'] = law_train['pre_content'].apply(tokenize_function)
law_train

,subdomain,pre_content,content,label,token
75220,판례,인수합병을 추진하는 입장에서 피고인에게 회사일을 거론할 수는 없었다는 정의 법정 진...,인수합병을 추진하는 입장에서 피고인에게 회사일을 거론할 수는 없었다는 정의 법정 진...,3,"[input_ids, token_type_ids, attention_mask, la..."
48955,세법,이는 차명거래를 한 데 대한 제재의 성격을 갖고 있다고 볼 수밖에 없다,이는 차명거래를 한 데 대한 제재의 성격을 갖고 있다고 볼 수밖에 없다.,2,"[input_ids, token_type_ids, attention_mask, la..."
44966,세법,행정 담당 공무원들은 년 담당하다가 자리를 옮기니 전문성을 갖출 시간이 부족하다,행정 담당 공무원들은 1∼2년 담당하다가 자리를 옮기니 전문성을 갖출 시간이 부족하다.,2,"[input_ids, token_type_ids, attention_mask, la..."
13568,민법,우리 민법은 이러한 일본민법의 태도를 따른 것이다,우리 민법은 이러한 일본민법의 태도를 따른 것이다.,0,"[input_ids, token_type_ids, attention_mask, la..."
92727,형법,채굴을 통해서 생성된 블록이 블록체인에 결합되기 위해서는 일정한 요건을 충족해야 한다,채굴을 통해서 생성된 블록이 블록체인에 결합되기 위해서는 일정한 요건을 충족해야 한다.,4,"[input_ids, token_type_ids, attention_mask, la..."
...,...,...,...,...,...
6265,민법,머신 러닝 기술을 활용하면 라는 미지의 입력 내용에 대해서도 컴퓨터는 이렇게 하면 ...,머신 러닝 기술을 활용하면 Z라는 미지의 입력 내용에 대해서도 컴퓨터는 이렇게 하면...,0,"[input_ids, token_type_ids, attention_mask, la..."
54886,세법,이에 따르면 시스템에 대한 사용자의 일반적 만족도는 동 시스템 구축 이전 자치단체별...,"이에 따르면, 시스템에 대한 사용자의 일반적 만족도는 동 시스템 구축 이전 자치단체...",2,"[input_ids, token_type_ids, attention_mask, la..."
76820,판례,근본적으로는 국내 해사 중재 활성화를 위한 기본 토대가 갖춰져 있지 않다는 점을 들...,"근본적으로는, 국내 해사 중재 활성화를 위한 기본 토대가 갖춰져 있지 않다는 점을 ...",3,"[input_ids, token_type_ids, attention_mask, la..."
860,민법,또한 는 증권거래세를 부과함에 따라 과세정보의 외부효과로 인하여 자원의 낭비를 감소...,"또한, AAA는 증권거래세를 부과함에 따라 과세정보의 외부효과로 인하여 자원의 낭비...",0,"[input_ids, token_type_ids, attention_mask, la..."


In [16]:
law_val['token'] = law_val['pre_content'].apply(tokenize_function)
law_val

,subdomain,pre_content,content,label,token
75721,판례,마지막으로 년에는 그동안 부진했던 증권 관련 집단소송 중 여러 건이 진행되었다,마지막으로 2018년에는 그동안 부진했던 증권 관련 집단소송 중 여러 건이 진행되었다.,3,"[input_ids, token_type_ids, attention_mask, la..."
80184,형법,사법 해석은 법률의 규정이나 입법 취지를 벗어나 해석할 수 없고 범죄 자산 몰수 특...,"사법 해석은 법률의 규정이나 입법 취지를 벗어나 해석할 수 없고, 범죄 자산 몰수 ...",4,"[input_ids, token_type_ids, attention_mask, la..."
19864,민법,이 사건 소가 전소에서 확정된 법률관계와 정반대의 모순되는 사항을 소송물로 하는 것...,이 사건 소가 전소에서 확정된 법률관계와 정반대의 모순되는 사항을 소송물로 하는 것...,0,"[input_ids, token_type_ids, attention_mask, la..."
76699,판례,법적 관점에서도 태아가 불법행위로 인한 손해배상청구권을 가진다,법적 관점에서도 태아가 불법행위로 인한 손해배상청구권을 가진다.,3,"[input_ids, token_type_ids, attention_mask, la..."
92991,형법,즉 생명권은 사람의 생존본능과 존재목적 고유한 존재가치에 바탕을 두고 있으므로 이는...,"즉, 생명권은 사람의 생존본능과 존재목적 고유한 존재가치에 바탕을 두고 있으므로 이...",4,"[input_ids, token_type_ids, attention_mask, la..."
...,...,...,...,...,...
32595,법률연구,대량 실업 사태와 임금 저하 현상이 속출하였고 년 노동법 체계에 대한 재검토가 불가...,"대량 실업 사태와 임금 저하 현상이 속출하였고, 1997년 노동법 체계에 대한 재검...",1,"[input_ids, token_type_ids, attention_mask, la..."
29313,법률연구,사건에서 영국 는 새로운 기술에 기초한 디자인의 경우 그러한 새로운 기술이 디자이...,"Dyson Ltd v Vax Ltd 사건에서, 영국 High Court는 새로운 기...",1,"[input_ids, token_type_ids, attention_mask, la..."
37862,법률연구,자유법론의 주장은 제정법과 법의 무흠결성이란 도그마 형식논리 법학에서의 의지 작용의...,"자유법론의 주장은 제정법과 법의 무흠결성이란 도그마, 형식논리, 법학에서의 의지 작...",1,"[input_ids, token_type_ids, attention_mask, la..."
53421,세법,는 구성요소를 분리하는 것이 어렵기 때문에 이익조정 및 회계기준과 세법의 기계적인...,BTD는 구성요소를 분리하는 것이 어렵기 때문에 이익조정 및 회계기준과 세법의 기계...,2,"[input_ids, token_type_ids, attention_mask, la..."


In [17]:
law_train['token'].iloc[0]

{'input_ids': [2, 20446, 4397, 21557, 12166, 7966, 14295, 26188, 16328, 8963, 9858, 17654, 4082, 15358, 12629, 4008, 8209, 19711, 14340, 4017, 12665, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_

In [19]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [20]:
mlm_train = law_train['token'].copy()
mlm_train.reset_index(drop=True, inplace=True)

mlm_val = law_val['token'].copy()
mlm_val.reset_index(drop=True, inplace=True)

In [21]:
mlm_train[100]

{'input_ids': [2, 21380, 8966, 8229, 10041, 13867, 12810, 4105, 9186, 8158, 16817, 4102, 7975, 7968, 10794, 13804, 4042, 18561, 9878, 7966, 12710, 9118, 4075, 13256, 4072, 4042, 11794, 26409, 18861, 8094, 26503, 8294, 8060, 4047, 2219, 8835, 903, 8556, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

### (2) train

In [22]:
training_args = TrainingArguments(output_dir='./fp_result/law_further_pretrained',
                                  evaluation_strategy="epoch",
                                  save_strategy="epoch",
                                  overwrite_output_dir=True,
                                  num_train_epochs=epochs,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  load_best_model_at_end=True,
                                  seed=seed)

trainer = Trainer(model=mlm_model,
                  args=training_args,
                  data_collator=data_collator,
                  train_dataset=mlm_train,
                  eval_dataset=mlm_val)

In [23]:
torch.cuda.empty_cache()

In [24]:
trainer.train()

C:\Users\delab_7\AppData\Roaming\Python\Python38\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 80000
  Num Epochs = 20
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 50000


Epoch,Training Loss,Validation Loss
1,2.778600,2.634197
2,2.588800,2.481555
3,2.445000,2.416952
4,2.368400,2.340377
5,2.261800,2.290115
6,2.189800,2.244710
7,2.132900,2.202764
8,2.055300,2.189559
9,1.995000,2.167366
10,1.952000,2.140527


***** Running Evaluation *****
  Num examples = 20000
  Batch size = 32
Saving model checkpoint to ./fp_result/law_further_pretrained\checkpoint-2500
Configuration saved in ./fp_result/law_further_pretrained\checkpoint-2500\config.json
Model weights saved in ./fp_result/law_further_pretrained\checkpoint-2500\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 20000
  Batch size = 32
Saving model checkpoint to ./fp_result/law_further_pretrained\checkpoint-5000
Configuration saved in ./fp_result/law_further_pretrained\checkpoint-5000\config.json
Model weights saved in ./fp_result/law_further_pretrained\checkpoint-5000\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 20000
  Batch size = 32
Saving model checkpoint to ./fp_result/law_further_pretrained\checkpoint-7500
Configuration saved in ./fp_result/law_further_pretrained\checkpoint-7500\config.json
Model weights saved in ./fp_result/law_further_pretrained\checkpoint-7500\pytorch_model.bin
***** Running Ev

TrainOutput(global_step=50000, training_loss=2.0268064184570314, metrics={'train_runtime': 37311.0822, 'train_samples_per_second': 42.883, 'train_steps_per_second': 1.34, 'total_flos': 2.4675300864e+17, 'train_loss': 2.0268064184570314, 'epoch': 20.0})

In [25]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 20000
  Batch size = 32


{'eval_loss': 1.9752689599990845,
 'eval_runtime': 139.914,
 'eval_samples_per_second': 142.945,
 'eval_steps_per_second': 4.467,
 'epoch': 20.0}